In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [3]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data("./mnist.npz")

In [4]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [8]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 126s 2ms/step - loss: 0.2675 - acc: 0.9186 - val_loss: 0.0551 - val_acc: 0.9823
Epoch 2/12
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0898 - acc: 0.9736 - val_loss: 0.0479 - val_acc: 0.9840
Epoch 3/12
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0659 - acc: 0.9800 - val_loss: 0.0328 - val_acc: 0.9899
Epoch 4/12
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0539 - acc: 0.9839 - val_loss: 0.0319 - val_acc: 0.9894
Epoch 5/12
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0474 - acc: 0.9859 - val_loss: 0.0276 - val_acc: 0.9922
Epoch 6/12
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0412 - acc: 0.9873 - val_loss: 0.0284 - val_acc: 0.9906
Epoch 7/12
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0369 - acc: 0.9889 - val_loss: 0.0280 - 

In [9]:
keras_model = "digit_clfr.h5"
keras.models.save_model(model,keras_model)

In [10]:
from tensorflow.contrib import lite

In [13]:
converter = lite.TFLiteConverter.from_keras_model_file(model_file=keras_model)

INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


In [ ]:
tflitemodel = converter.convert()
